In [ ]:
# @title Install HF hub First
!pip install huggingface-hub

In [ ]:
# @title Upload Config
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

# @markdown Login to Huggingface Hub
# @markdown > Get **your** huggingface `WRITE` token [here](https://huggingface.co/settings/tokens)
write_token = ""  # @param {type:"string"}
# @markdown Fill this if you want to upload to your organization, or just leave it empty.
orgs_name = ""  # @param{type:"string"}
# @markdown If your model/dataset repo does not exist, it will automatically create it.
model_name = ""  # @param{type:"string"}
dataset_name = ""  # @param{type:"string"}
make_private = True  # @param{type:"boolean"}

def authenticate(write_token):
    login(write_token, add_to_git_credential=True)
    api = HfApi()
    return api.whoami(write_token), api


def create_repo(api, user, orgs_name, repo_name, repo_type, make_private=False):
    global model_repo
    global datasets_repo

    if orgs_name == "":
        repo_id = user["name"] + "/" + repo_name.strip()
    else:
        repo_id = orgs_name + "/" + repo_name.strip()

    try:
        validate_repo_id(repo_id)
        api.create_repo(repo_id=repo_id, repo_type=repo_type, private=make_private)
        print(f"{repo_type.capitalize()} repo '{repo_id}' didn't exist, creating repo")
    except HfHubHTTPError as e:
        print(f"{repo_type.capitalize()} repo '{repo_id}' exists, skipping create repo")

    if repo_type == "model":
        model_repo = repo_id
        print(f"{repo_type.capitalize()} repo '{repo_id}' link: https://huggingface.co/{repo_id}\n")
    else:
        datasets_repo = repo_id
        print(f"{repo_type.capitalize()} repo '{repo_id}' link: https://huggingface.co/datasets/{repo_id}\n")

user, api = authenticate(write_token)

if model_name:
    create_repo(api, user, orgs_name, model_name, "model", make_private)
if dataset_name:
    create_repo(api, user, orgs_name, dataset_name, "dataset", make_private)


In [ ]:
# @title ### Upload Data Ver Asahina

from huggingface_hub import HfApi
from pathlib import Path
api = HfApi()

# @markdown This will be uploaded to model repo
folder_loc = "/content/drive/MyDrive/Vae" # @param {type :"string"}
path_in_repo = "Vae"  # @param {type :"string"}

# @markdown Other Information
commit_message = "Vae V1"  # @param {type :"string"}

if not commit_message:
    commit_message = f"feat: upload Data To HF"

def upload_to_hf(folder_loc, is_folder, is_config):
    path_obj = Path(folder_loc)
    trained_model = path_obj.parts[-1]

    if path_in_repo:
        trained_model = path_in_repo

    print(f"Uploading {trained_model} to https://huggingface.co/{model_repo}")
    print("Please wait...")

    if is_folder:
        api.upload_folder(
            folder_path=folder_loc,
            path_in_repo=trained_model,
            repo_id=model_repo,
            commit_message=commit_message,
            ignore_patterns=".ipynb_checkpoints",
        )
        print(f"Upload success, located at https://huggingface.co/{model_repo}/tree/main\n")
    else:
        api.upload_file(
            path_or_fileobj=folder_loc,
            path_in_repo=trained_model,
            repo_id=model_repo,
            commit_message=commit_message,
        )
        print(f"Upload success, located at https://huggingface.co/{model_repo}/blob/main/{trained_model}\n")

def upload():
    is_model_file = folder_loc.endswith((".ckpt", ".safetensors", ".pt"))
    upload_to_hf(folder_loc, not is_model_file, False)

upload()